In [2]:
from Sindonews import Sindonews #class news Scrapper
from datetime import timedelta, date, datetime
from IPython.display import clear_output

In [3]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [4]:
obj = Sindonews()
end_date = date(2018, 7, 24)
start_date = date(2018, 5, 28)
init_page = 10
init_offset = 90

In [ ]:
for single_date in daterange(start_date, end_date):
    clear_output()
    print(single_date.strftime("%Y-%m-%d"))
    obj.getAllBerita([], init_page, 0, init_offset, single_date.strftime("%Y-%m-%d"))
    init_page = 1
    init_offset = 0

2018-05-29
page  1
https://index.sindonews.com/index/0/0?t=2018-05-29
https://soccer.sindonews.com/read/1309993/59/iago-aspas-hibur-morata-setelah-terlempar-dari-skuat-matador-1527603042
Insert berita  Iago Aspas Hibur Morata Setelah Terlempar dari Skuat Matador
masuk
https://daerah.sindonews.com/read/1310011/23/kartini-perindo-di-jombang-bagi-beras-untuk-warga-miskin-1527611755
Insert berita  Kartini Perindo di Jombang Bagi Beras untuk Warga Miskin
masuk
https://international.sindonews.com/read/1310010/43/serangan-mortir-besar-besaran-dari-gaza-bikin-warga-israel-panik-1527611669
Insert berita  Serangan Mortir Besar-besaran dari Gaza Bikin Warga Israel Panik
masuk
https://lifestyle.sindonews.com/read/1309933/187/mariah-carey-jual-cincin-tunangan-1527587774
Insert berita  Mariah Carey Jual Cincin Tunangan
masuk
https://daerah.sindonews.com/read/1310009/22/dua-atm-bank-pemerintah-dibobol-maling-1527610005
Insert berita  Dua ATM Bank Pemerintah Dibobol Maling
masuk
https://ekbis.sindonew

masuk
https://lifestyle.sindonews.com/read/1309869/158/captain-america-dan-black-widow-dapat-peran-besar-di-avengers-4-1527570125
Insert berita  Captain America dan Black Widow Dapat Peran Besar di Avengers 4
masuk
https://international.sindonews.com/read/1309965/40/menlu-filipina-duterte-selalu-siap-perang-dengan-china-1527596507
Insert berita  Menlu Filipina: Duterte Selalu Siap Perang dengan China
masuk
https://nasional.sindonews.com/read/1309962/12/tokoh-muda-muhammadiyah-uang-bukan-lagi-tujuan-hidup-pejabat-bpip-1527595675
Insert berita  Tokoh Muda Muhammadiyah: Uang Bukan Lagi Tujuan Hidup Pejabat BPIP
masuk
https://daerah.sindonews.com/read/1309960/23/rumah-kos-di-surabaya-terbakar-8-orang-tewas-di-lokasi-1527595485
Insert berita  Rumah Kos di Surabaya Terbakar, 8 Orang Tewas di Lokasi
masuk
https://metro.sindonews.com/read/1309964/171/di-hadapan-ribuan-umat-budha-anies-ajak-semua-umat-bersatu-1527596016
Insert berita  Di Hadapan Ribuan Umat Budha, Anies Ajak Semua Umat Bersatu


masuk
https://international.sindonews.com/read/1309916/43/maroko-kirim-tim-medis-ke-jalur-gaza-1527583795
Insert berita  Maroko Kirim Tim Medis ke Jalur Gaza
masuk
https://daerah.sindonews.com/read/1309909/21/arus-mudik-di-jabar-diprediksi-naik-5-persen-1527581773
Insert berita  Arus Mudik di Jabar Diprediksi Naik 5 Persen
masuk
https://nasional.sindonews.com/read/1309913/12/jokowi-sebut-gaji-bpip-melalui-analisa-kemenpan-rb-dan-kemenkeu-1527582493
Insert berita  Jokowi Sebut Gaji BPIP Melalui Analisa Kemenpan RB dan Kemenkeu
masuk
https://sports.sindonews.com/read/1309914/47/tersingkir-dini-vika-tak-bisa-bawa-hasil-latihan-ke-pertandingan-1527582631
Insert berita  Tersingkir Dini, Vika Tak Bisa Bawa Hasil Latihan ke Pertandingan
masuk
page  10
https://index.sindonews.com/index/0/90?t=2018-05-29
https://lifestyle.sindonews.com/read/1309848/155/4-minuman-yang-bisa-redakan-asam-lambung-naik-saat-puasa-1527564154
Insert berita  4 Minuman yang Bisa Redakan Asam Lambung Naik Saat Puasa
masu